# Testing Agentic Planning with ControlFlow

In [2]:
from datetime import datetime
from typing import List, Literal, Optional

import controlflow as cf
from dotenv import load_dotenv
from pydantic import BaseModel, Field

load_dotenv()

True

In [3]:
reply = cf.run(
    "Chunk a project into smaller tasks",
    context=dict(project="Get a job as a genAI engineer"),
)

print(reply)

Output()

1. Research GenAI Industry Roles: \n   a. Understand different positions available for GenAI engineers.\n   b. Note required skills and qualifications.\n   c. Identify key companies and sectors hiring GenAI engineers.\n\n2. Skills Development: \n   a. Learn essential programming languages (e.g., Python, Java).\n   b. Gain proficiency in AI frameworks and tools (e.g., TensorFlow, PyTorch).\n   c. Understand machine learning algorithms and data analysis.\n   d. Build a portfolio of projects to showcase skills.\n\n3. Networking and Professional Development: \n   a. Join relevant industry groups and forums.\n   b. Attend conferences, webinars, and workshops.\n   c. Connect with professionals and mentors in the field.\n   d. Build a strong LinkedIn profile.\n\n4. Application Process: \n   a. Tailor resume and cover letters for GenAI roles.\n   b. Prepare for technical interviews and coding assessments.\n   c. Research and apply for open positions.\n   d. Follow up with applications and inte

In [4]:
# Create a specialized agent
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions="You are a project planning expert using the Getting Things Done \
        (GTD) methodology. When given a project or task, chunk it down into a \
        hierarchical tree of subtasks.",
)


# Set up a ControlFlow task to classify emails
tasks = cf.run(
    "Chunk the project into a hierarchical task tree",
    # result_type=TaskList,
    agents=[chunker],
    context=dict(project="Get a job as a genAI engineer"),
)

print(tasks)

Output()

Get a job as a genAI engineer:  
- Research the field of genAI  
  - Understand the job market  
  - Identify key companies hiring genAI engineers  
  - Research job requirements and skills  
- Build key skills  
  - Take relevant online courses  
  - Obtain certifications  
  - Practice through projects and portfolio-building  
- Update resume and LinkedIn profile  
  - Tailor resume to highlight genAI skills  
  - Update LinkedIn with new skills and experiences  
- Apply for jobs  
  - Create a list of target companies  
  - Craft personalized cover letters  
  - Submit applications  
- Prepare for interviews  
  - Practice common interview questions  
  - Conduct mock interviews  
- Network with professionals in the field  
  - Attend industry conferences  
  - Join genAI online communities  
  - Connect with professionals on LinkedIn


In [5]:
# Create a specialized agent
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions="You are a project planning expert using the Getting Things Done \
        (GTD) methodology. When given a project or task, chunk it down into a \
        hierarchical tree of subtasks.",
)


class ChunkedTask(BaseModel):
    id: str = Field(description="Unique identifier for the task")
    parent_id: Optional[str] = Field(
        None, description="ID of parent task, null if root"
    )
    title: str = Field(description="Title that captures the task's action/outcome")
    created_at: datetime = Field(default_factory=datetime.utcnow)


class TaskList(BaseModel):
    tasks: List["ChunkedTask"] = Field(default_factory=list)


# Set up a ControlFlow task to classify emails
chunk_task = cf.Task(
    objective="Chunk the project into a hierarchical task tree",
    result_type=TaskList,
    agents=[chunker],
    context=dict(project="Get a job as a genAI engineer"),
)

tasks = chunk_task.run()

print(tasks)

Output()

tasks=[ChunkedTask(id='1', parent_id=None, title='Get a job as a genAI engineer', created_at=datetime.datetime(2025, 2, 12, 14, 33, 23, 479350)), ChunkedTask(id='1.1', parent_id='1', title='Update resume', created_at=datetime.datetime(2025, 2, 12, 14, 33, 23, 479350)), ChunkedTask(id='1.1.1', parent_id='1.1', title='List relevant skills and experiences', created_at=datetime.datetime(2025, 2, 12, 14, 33, 23, 479350)), ChunkedTask(id='1.1.2', parent_id='1.1', title='Format resume according to industry standards', created_at=datetime.datetime(2025, 2, 12, 14, 33, 23, 479350)), ChunkedTask(id='1.1.3', parent_id='1.1', title='Proofread and edit resume', created_at=datetime.datetime(2025, 2, 12, 14, 33, 23, 479350)), ChunkedTask(id='1.2', parent_id='1', title='Build portfolio', created_at=datetime.datetime(2025, 2, 12, 14, 33, 23, 479350)), ChunkedTask(id='1.2.1', parent_id='1.2', title='Select projects to showcase', created_at=datetime.datetime(2025, 2, 12, 14, 33, 23, 479350)), ChunkedTask

## Multi-Agent Solution
The flow we're eying atm would be this:
```
A: input P -> B [find Ps without NA children]
B -> C{Any found?}
C -> |yes| D[generate children for these Ps]
D -> E[Classify new children as P or NA]
E -> D
C -> |no| F[Check if each P's direct children achieve its outcome]
F -> |Incomplete Ps| D
F -> |all complete| G[Final Task Tree]
```

But let's build towards this gradually.






In [42]:
def print_task_tree(tasks: TaskList, parent_id: str = "1", indent: int = 0):
    """Print tasks in a hierarchical tree format.

    Args:
        tasks: TaskList containing all tasks
        parent_id: ID of parent task to start from (defaults to "1")
        indent: Current indentation level
    """
    # Find tasks with this parent
    current_level = [t for t in tasks.tasks if t.parent_id == parent_id]

    # Print each task at this level
    for task in current_level:
        print("    " * indent + f"- {task.title}")
        print_task_tree(tasks, task.id, indent + 1)

### V0.1 - 1 agent and 1 task to chunk a project

In [25]:
class TaskPNA(BaseModel):
    id: str = Field(description="unique identifier for the task")
    title: str = Field(description="A short title/description of the task")
    parent_id: Optional[str] = Field(
        None, description="ID of this task's parent task, null if root"
    )


class TaskList(BaseModel):
    tasks: List[TaskPNA] = Field(default_factory=list)

In [53]:
# Create a specialized agent
chunker_instructions = """
You are a Getting Things Done project management expert, specialized into
subdividing big tasks into smaller ones.
"""
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions=chunker_instructions,
)


@cf.flow
def chunk_flow_v0_1(project: str):
    # Initialize with root project
    root_task_id = "1"
    tasks = TaskList(tasks=[TaskPNA(id=root_task_id, title=project, parent_id=None)])

    with cf.Task(
        "Generate hyrarchical task tree for this project",
        result_type=TaskList,
        agents=[chunker],
    ) as main_task:
        new_tasks = cf.run(
            """
            Break given task (with id {task_id}) into subtasks.
            Ensure that any subtask saves the task_id of its parent task as its
            parent_id.
            Once you split the task into subtasks, mark this task as complete and
            use the main task tool to mark that as complete.
            """,
            context=dict(
                task_id=root_task_id,
                tasks=tasks,
            ),
            agents=[chunker],
            result_type=TaskList,
            tools=[main_task.get_success_tool()],
        )

        tasks = TaskList(tasks=tasks.tasks + new_tasks.tasks)

    return tasks


tasks = chunk_flow_v0_1("Get a job as a genAI engineer")

print(tasks)

Output()

tasks=[TaskPNA(id='1', title='Get a job as a genAI engineer', parent_id=None), TaskPNA(id='1.1', title='Update resume to include relevant experience', parent_id='90f226f7'), TaskPNA(id='1.2', title='Research companies hiring genAI engineers', parent_id='90f226f7'), TaskPNA(id='1.3', title='Prepare for technical interviews by practicing coding', parent_id='90f226f7'), TaskPNA(id='1.4', title='Apply for jobs by submitting applications online', parent_id='90f226f7'), TaskPNA(id='1.5', title='Network with professionals in the genAI field', parent_id='90f226f7')]


In [51]:
print(tasks)

tasks=[TaskPNA(id='1', title='Get a job as a genAI engineer', parent_id=None), TaskPNA(id='2', title='Update resume', parent_id='1'), TaskPNA(id='3', title='Prepare portfolio samples', parent_id='1'), TaskPNA(id='4', title='Apply to job postings', parent_id='1'), TaskPNA(id='5', title='Prepare for interviews', parent_id='1')]


In [54]:
print([f"{t.id}: {t.title} (parent: {t.parent_id})" for t in tasks.tasks])

['1: Get a job as a genAI engineer (parent: None)', '1.1: Update resume to include relevant experience (parent: 90f226f7)', '1.2: Research companies hiring genAI engineers (parent: 90f226f7)', '1.3: Prepare for technical interviews by practicing coding (parent: 90f226f7)', '1.4: Apply for jobs by submitting applications online (parent: 90f226f7)', '1.5: Network with professionals in the genAI field (parent: 90f226f7)']


In [55]:
print_task_tree(tasks)

### V1.0 - multiple agents etc (WIP - not working)

##### Set up data models, agents, and tasks

In [25]:
class TaskPNA(BaseModel):
    id: str
    title: str = Field(
        description="""
        For Projects: The desired outcome state (e.g., "garage cleaned and organized")
        For Next Actions: The concrete action (e.g., "sweep garage floor")
    """
    )
    parent_id: Optional[str] = Field(None, description="parent task ID, null if root")
    classification: Optional[Literal["Project", "Next Action"]] = Field(
        None, description="Whether this task is a Project or Next Action"
    )


class TaskList(BaseModel):
    tasks: List[TaskPNA] = Field(default_factory=list)


class TaskClassification(BaseModel):
    task_id: str
    classification: Literal["Project", "Next Action"]


# Agents
chunker = cf.Agent(
    name="Project Chunker",
    model="openai/gpt-4o-mini",
    instructions="""You are a Getting Things Done expert, specialized in subdividing 
        Projects into Next Actions or sub-Projects.
        
        When describing a Project, always phrase it as a completed outcome state 
        (e.g., "kitchen fully renovated", "report completed and approved").
        
        When describing a Next Action, always phrase it as a concrete physical action 
        (e.g., "call contractor at 555-0123", "draft first section of report").
        
        Each task description should be clear enough that someone else could understand 
        exactly what needs to be done or what outcome needs to be achieved.""",
)

reviewer = cf.Agent(
    name="Task Reviewer",
    model="openai/gpt-4o-mini",
    instructions="""You are a Getting Things Done expert, categorizing tasks as either 
        Projects or Next Actions.
        
        A Next Action:
        - Is a single, physical action
        - Can be done in one sitting
        - Has no prerequisites
        - Uses an action verb
        - Is very specific (e.g., "email John about budget" vs "contact John")
        
        A Project:
        - Requires multiple actions to complete
        - Is described as a completed outcome
        - Uses past tense to describe the desired state
        - Will likely need planning or coordination""",
)

completeness_checker = cf.Agent(
    name="Completeness Checker",
    model="openai/gpt-4o-mini",
    instructions="""You are a Getting Things Done expert, specialized in verifying if a 
        set of subtasks is sufficient to achieve their parent task's outcome.
        
        For each Project, evaluate if its direct children (both Next Actions and 
        sub-Projects) collectively will achieve the Project's stated outcome.
        
        Example:
        Project: "garage cleaned and organized"
        Subtasks:
        1. "remove items from garage"
        2. "sort items into keep/donate piles"
        3. "sweep garage floor"
        
        This is incomplete because it's missing:
        - Disposing of non-kept items
        - Organizing kept items back into garage
        - Potentially cleaning walls/surfaces""",
)

# Tasks
chunk_task = cf.Task(
    objective="Generate child tasks for the given Project",
    result_type=TaskList,
    agents=[chunker],
)

review_task = cf.Task(
    objective="Classify each unclassified task as either a Project or Next Action",
    result_type=TaskList,
    agents=[reviewer],
)

completeness_task = cf.Task(
    objective="Check if the Project's direct children will achieve its outcome \
        and return the IDs of incomplete Projects as a list of strings.",
    result_type=List[str],  # IDs of incomplete Projects
    agents=[completeness_checker],
)

In [26]:
def find_projects_without_nas(tasks: TaskList) -> List[TaskPNA]:
    """Find all Projects that don't have any Next Action children.

    Args:
        tasks: TaskList containing all tasks

    Returns:
        List of Project tasks that have no Next Action children
    """
    result = []

    for task in tasks.tasks:
        if task.classification != "Project":
            continue

        # Find all children of this task
        children = [t for t in tasks.tasks if t.parent_id == task.id]
        # Check if any children are Next Actions
        na_children = [c for c in children if c.classification == "Next Action"]

        if not na_children:
            result.append(task)

    return result

#### Define the flow

In [29]:
@cf.flow
def chunking_flow(project: str):
    # Initialize with root project
    tasks = TaskList(tasks=[TaskPNA(id="1", title=project, classification="Project")])

    with cf.Task(
        "Break down project into a complete task tree",
        result_type=TaskList,
        agents=[chunker, reviewer, completeness_checker],
    ) as main_task:
        while main_task.is_incomplete():
            # Find Ps without NA children
            projects_without_nas = find_projects_without_nas(tasks)

            if projects_without_nas:
                # Generate and classify new tasks
                cf.run(
                    "Generate children for projects without Next Actions",
                    context=dict(
                        parent_tasks=projects_without_nas, existing_tasks=tasks
                    ),
                    agents=[chunker],
                    result_type=TaskList,
                )

                cf.run(
                    "Classify new tasks",
                    context=dict(tasks=tasks),
                    agents=[reviewer],
                    result_type=TaskList,
                )
            else:
                # Check completeness
                cf.run(
                    "Check if all Projects are complete",
                    context=dict(tasks=tasks),
                    agents=[completeness_checker],
                    result_type=List[str],
                )

    return main_task.result

In [31]:
cf.settings.debug_messages = True

chunking_flow("Get a job as a genAI engineer")

Output()

Output()

13:04:26.601 | ERROR   | Flow run 'wisteria-porpoise' - Encountered exception during execution: ValueError('1 task failed: - Task #775c16dd ("Classify new tasks"): No reasons for failure exist.')
Traceback (most recent call last):
  File "c:\Users\jonat\Documents\code\taskchunker-backend\notebooks\.venv-notebooks\Lib\site-packages\prefect\flow_engine.py", line 764, in run_context
    yield self
  File "c:\Users\jonat\Documents\code\taskchunker-backend\notebooks\.venv-notebooks\Lib\site-packages\prefect\flow_engine.py", line 1372, in run_flow_sync
    engine.call_flow_fn()
  File "c:\Users\jonat\Documents\code\taskchunker-backend\notebooks\.venv-notebooks\Lib\site-packages\prefect\flow_engine.py", line 784, in call_flow_fn
    result = call_with_parameters(self.flow.fn, self.parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonat\Documents\code\taskchunker-backend\notebooks\.venv-notebooks\Lib\site-packages\prefect\utilities\callables.py", line 208, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "c:\Users\jonat\Documents\code\taskchunker-backend\notebooks\.venv-notebooks\Lib\site-packages\controlflow\decorators.py", line 113, in wrapper
    return fn(*wrapper_args, **wrapper_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jonat\AppData\Local\Temp\ipykernel_16300\3618565082.py", line 34, in chunking_flow
    cf.run(
  File "c:\Users\jonat\Documents\code\taskchunker-backend\notebooks\.venv-notebooks\Lib\site-packages\controlflow\run.py", line 147, in run
    results = run_tasks(
              ^^^^^^^^^^
  File "c:\Users\jonat\Documents\code\taskchunker-backend\notebooks\.venv-notebooks\Lib\site-packages\controlflow\run.py", line 59, in run_tasks
    raise ValueError(
ValueError: 1 task failed: - Task #775c16dd ("Classify new tasks"): No reasons for failure exist.

13:04:26.623 | ERROR   | Flow run 'wisteria-porpoise' - Finished in state Failed('Flow run encountered an exception: ValueError: 1 task failed: - Task #775c16dd ("Classify new tasks"): No reasons for failure exist.')

ValueError: 1 task failed: - Task #775c16dd ("Classify new tasks"): No reasons for failure exist.